# Key Distribution - BB84 Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Key Distribution BB84 kata](./KeyDistribution_BB84.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Key Distribution - BB84 Kata (and this workbook):
1. Basic Qubit Operations
2. The concept of measurement

### Background

**What is problem of Key Distribution?**
In Symmetric Key Cryptography, Alice and Bob: the two parties who wishes to communicate in a secured manner, are required to share a key using which encryption and decryption would work. Alice would calculate the cipher text using this key and the message to be encrypted. Bob, on the other hand would require the same key to decrypt the cipher text into plain text for understanding. Thus, the problem of secure communication can be reduced to the problem to sharing a *key*.

In classical computing, Assymmetric Cryptographic techniques like RSA, Elliptical Curve Cryptography were used to perform encryption. These techniques are based on the fact that it is not possible to solve the problem of prime factorization or discrete logarithm in polynomial time. Both of these can easily be solved using Quantum Shor's Algorithm and thus it is required to visit back the problem of Key Distribution.

**Quantum Key Distribution**
Quantum Key Distribution can be thought of as a new tool in the toolbox of a Cryptographer. It nowhere eliminates the need for other cryptographic steps like authentication, but rather help build a system with new security properties. 

Today we will learn about the BB84 Technique of Quantum Key Distribution which was maded after its scientists **B**ennett and **B**rassard in the year 19**84**. It is proven to be uncoditionally secure in theory and thus remain a strong foundatation of Quantum Key Distribution techniques, even today.

## Part I. Preparation

While preparing qubits in Quantum Key Distribution, Alice is required to choose a random bit string. For each bit, she has to choose the bases in which it should be measured so that in case Eve wants to read this data, she would be in equal probability of getting the correct or the wrong answer.

We are required to choose two bases to achieve the the same. In pratical implementation, this refers to choosing different types of polarization. Each photon is either polarized in Rectilinear Basis or in Diagonal Basis.

In the first type of basis (referred above as Rectilinear Basis or Computational Basis), Alice would prepares the states $|0\rangle$ and $|1\rangle$ where $|0\rangle$ stands for bit value `0` and $|1\rangle$ stands for bit value `1`. The second basis (referred above as the diagonal basis or Hadamard Basis) represents bit value `0` as the states $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$, and bit value `1` as $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$.

### <a name="diagonal_basis"></a>  Task 1.1. Diagonal Basis

Try your hand at converting qubits from the computational basis to the diagonal basis.

**Input:** $N$ qubits (stored in an array of length $N$). Each qubit is either in $|0\rangle$ or in $|1\rangle$ state.

**Goal:**  Convert the qubits to the diagonal basis: 
* if `qs[i]` was in state $|0\rangle$, it should be transformed to $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$,
* if `qs[i]` was in state $|1\rangle$, it should be transformed to $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$.

### Solution

This task is similar to the one mentioned in [task 1.9 of Superposition kata](./../Superposition/Superposition.ipynb#superposition-of-all-basis-vectors). It is great if you took the hint from the explaination above otherwise, this task consists of applying Hadamard Gate to every Qubit resulting in $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$ and $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$ state based on the input provided as $|0\rangle$ or $|1\rangle$, respectively. 


Let us start with a basic example where input string is a string of zeros, $|00....0\rangle$.
\begin{align*}
   H|0\rangle &= \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
   H|0\rangle \otimes H|0\rangle &= \frac{1}{\sqrt2} \big(|0\rangle + |1\rangle\big) \otimes \frac{1}{\sqrt2} \big(|0\rangle + |1\rangle\big)\\   
               &= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big)\\
   H|0\rangle \otimes H|0\rangle \otimes H|0\rangle &= \frac{1}{\sqrt{2^2}}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
               &= \frac{1}{\sqrt{2^3}}\big(|000\rangle + |001\rangle + |010\rangle+ |100\rangle+ |110\rangle + |101\rangle+ |011\rangle+ |111\rangle\big)\\
    \underset{N}{\underbrace{H|0\rangle \otimes \dots \otimes H|0\rangle}} 
               &= \frac{1}{\sqrt{2^{N-1}}}  \big( |\underset{N-1}{\underbrace{0 \cdots 0}}\rangle + \cdots + |\underset{N-1}{\underbrace{1 \cdots 1}}\rangle \big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big) = \\
               &= \frac{1}{\sqrt{2^N}} \big( |\underset{N}{\underbrace{0 \cdots 0}}\rangle + \cdots + |\underset{N}{\underbrace{1 \cdots 1}}\rangle \big)\\    
\end{align*}

Using the same principles, we can consider the case where we have a string of ones, $|11....1\rangle$.
\begin{align*}
   H|1\rangle &= \frac{1}{\sqrt2}\big(|0\rangle - |1\rangle\big)\\
   H|1\rangle \otimes H|1\rangle &= \frac{1}{\sqrt2} \big(|0\rangle - |1\rangle\big) \otimes \frac{1}{\sqrt2} \big(|0\rangle - |1\rangle\big)\\   
               &= \frac{1}{\sqrt{2^2}}\big(|00\rangle - |01\rangle- |10\rangle+ |11\rangle\big)\\
   H|1\rangle \otimes H|1\rangle \otimes H|1\rangle &= \frac{1}{\sqrt{2^2}}\big(|00\rangle - |01\rangle- |10\rangle+ |11\rangle\big) \otimes \frac{1}{\sqrt2}\big(|0\rangle - |1\rangle\big)\\
               &= \frac{1}{\sqrt{2^3}}\big(|000\rangle - |001\rangle - |010\rangle- |100\rangle+ |110\rangle + |101\rangle+ |011\rangle- |111\rangle\big)\\
    \underset{N}{\underbrace{H|1\rangle \otimes \dots \otimes H|1\rangle}} 
               &= \frac{1}{\sqrt{2^{N-1}}}  \big( |\underset{N-1}{\underbrace{0 \cdots 0}}\rangle + \cdots + {(-1)^{N-1}} |\underset{N-1}{\underbrace{1 \cdots 1}}\rangle \big) \otimes \frac{1}{\sqrt2}\big(|0\rangle - |1\rangle\big) = \\
               &= \frac{1}{\sqrt{2^N}} \big( |\underset{N}{\underbrace{0 \cdots 0}}\rangle + \cdots + {(-1)^{N}}|\underset{N}{\underbrace{1 \cdots 1}}\rangle \big)\\    
\end{align*}

Combining the two equation, it is easy to calculate the H operation on $|0\rangle$ and $|1\rangle$.
\begin{align*}
   H|0\rangle \otimes H|1\rangle &= \frac{1}{\sqrt2} \big(|0\rangle + |1\rangle\big) \otimes \frac{1}{\sqrt2} \big(|0\rangle - |1\rangle\big)\\   
               &= \frac{1}{\sqrt{2^2}}\big(|00\rangle - |01\rangle+ |10\rangle- |11\rangle\big)\\
   H|0\rangle \otimes H|1\rangle \otimes H|0\rangle &= \frac{1}{\sqrt{2^2}}\big(|00\rangle - |01\rangle+ |10\rangle- |11\rangle\big) \otimes \frac{1}{\sqrt2}\big(|0\rangle + |1\rangle\big)\\
               &= \frac{1}{\sqrt{2^3}}\big(|000\rangle + |001\rangle - |010\rangle+ |100\rangle- |110\rangle + |101\rangle- |011\rangle- |111\rangle\big)\\
    \underset{N}{\underbrace{H|0\rangle \otimes \dots \otimes H|1\rangle}} 
               &= \frac{1}{\sqrt{2^{N-1}}}  \big( |\underset{N-1}{\underbrace{0 \cdots 0}}\rangle + \cdots + {(-1)^{N-1}} |\underset{N-1}{\underbrace{1 \cdots 1}}\rangle \big) \otimes \frac{1}{\sqrt2}\big(|0\rangle - |1\rangle\big) = \\
               &= \frac{1}{\sqrt{2^N}} \big( |\underset{N}{\underbrace{0 \cdots 0}}\rangle + \cdots + {(-1)^{N}}|\underset{N}{\underbrace{1 \cdots 1}}\rangle \big)\\    
\end{align*}

We can either iterate over all the qubits, one by one or use the [ApplyToEachA](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.applytoeacha) function to get the desired output.

In [ ]:
%kata T11_DiagonalBasis

operation DiagonalBasis (qs : Qubit[]) : Unit {    
    /*
        You can also use the following:
        for q in qs {
            H(q);
        }
    */
    
    //Otherwise use the inbuilt function of Q#
    ApplyToEachA(H,qs);
}

[Return to task 1.1 of the KeyDistribution_BB84 kata.](./KeyDistribution_BB84.ipynb#Task-1.1-diagonal-basis)

### <a name="equal_superposition"/> Task 1.2. Equal superposition

 
**Input**: A qubit in the $|0\rangle$ state.

**Goal**:  Change the qubit state to a superposition state that has equal probabilities of measuring 0 and 1. 

> Note that this is not the same as keeping the qubit in the $|0\rangle$ state with 50% probability and converting it to the $|1\rangle$ state with 50% probability!

### Solution
This task is similar to [Task 1.1 of Superposition kata](./../Superposition/Superposition.ipynb#plus-state) or [Task 1.2 of Superposition kata](./../Superposition/Superposition.ipynb#minus-state). 

This can also be thought as a specific use case of the previous step with **N** = 1.

If the equal superposition is required with a know state of $|0\rangle$, Hadamard gate should be an undoubtable choice. We can create any one of the following states to get the desired results from this function.
1. Plus State : $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$
2. Minus State : $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$.

Both the states with provide equal superposition of both $|0\rangle$ and $|1\rangle$.

In [ ]:
%kata T12_EqualSuperposition 

operation EqualSuperposition (q : Qubit) : Unit {
    
    // Straight forward solution to get the desired results. This would give plus state.
    H(q);
    
    //Alternatively, we can also use the following. This would give minus state.
    //X(q);
    //H(q);
    
}

[Return to task 1.2 of the Key Distribution kata.](././KeyDistribution_BB84.ipynb#Task-1.2-equal-superposition)